In [7]:
import requests  # requests 라이브러리를 사용하여 HTTP 요청을 보냄  
import os
import re
import dotenv

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")

# Azure OpenAI 서비스의 엔드포인트 URL  
endpoint = OPENAI_ENDPOINT
# API 키를 설정. 반드시 보안을 유지해야 하며, 실제 개발 환경에서는 환경 변수에 저장하는 것이 좋음  
api_key = OPENAI_API_KEY

# Azure AI Search 서비스의 엔드포인트 URL  
ai_search_endpoint = AZURE_SEARCH_ENDPOINT  
# Azure AI Search 서비스의 API 키  
ai_search_api_key = AZURE_SEARCH_KEY
# Azure AI Search에서 사용할 인덱스 이름  
ai_search_index = "hospital-index-lucene"
ai_search_semantic = "hospital-semantic-lucene"

def request_gpt(prompt):
    
    # HTTP 요청에 필요한 헤더 설정  
    # Content-Type은 요청 본문이 JSON 형식임을 나타냄  
    # api-key는 Azure OpenAI 서비스의 인증에 사용됨  
    headers = {  
        "Content-Type": "application/json",  
        "api-key": api_key
    }
    
    # HTTP 요청의 본문 데이터  
    # messages 리스트에는 대화의 역할(role)과 내용(content)이 포함됨  
    # "system" 역할은 모델의 행동을 정의하며, 여기서는 "남해 여행 전문가"로 설정  
    # "user" 역할은 사용자가 입력한 메시지를 나타냄  
    body = {  
        "messages": [  
            {  
                "role": "system",  # 시스템 역할: 모델의 컨텍스트 설정  
                "content": f'''너는 성남시 동물 병원을 잘 아는 전문가야.\n\n
                참조 데이터 "경기도 성남시 분당구 구미동 205-1번지 오성프라자 113호 경기도 성남시 분당구 미금로 48, 
                오성프라자 113호 (구미동) 나라 동물병원 분당구 2011-08-16 구미동 031-712-0707"라고 가정했을때 각각 "주소, 소재지도로명주소, 병원이름, 구, 개업일, 동, 전화번호"야.\n\n
                병원이름은 항상 참조 데이터 안에 ) 문자 뒤에 오는거야.
                대답할 때 병원이름은 반드시 포함해야해. 그리고 병원이름은 반드시 틀리면 안돼. 소재지도로명주소는 대답하지마.'''  # 모델에게 특정 역할을 부여  
            },  
            {  
                "role": "user",  # 사용자 역할: 사용자의 요청  
                "content": prompt  # 사용자가 알고 싶은 내용 (남해 관광지 3곳)  
            }  
        ],  
        "temperature": 0.0,  # 응답의 창의성 정도를 조절 (0.0은 보수적, 1.0은 매우 창의적)  
        "top_p": 0.95,  # 확률 분포에서 상위 p%를 선택하여 응답 생성  
        "max_tokens": 800,  # 응답에 사용할 최대 토큰 수 (토큰은 단어 및 기호 단위)  
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 5,
                "top_n_documents": 10,
                "authentication": {
                    "type": "api_key",
                    "key": ai_search_api_key
                },
                "key": ai_search_api_key,
                
            }
            }
        ],
    }  
    
    # POST 요청을 보내고 응답 받기  
    response = requests.post(endpoint, headers=headers, json=body)  
    print(response)

    if response.status_code == 200:

        # 응답을 JSON 형식으로 파싱  
        response_json = response.json()  
        
        # 모델이 생성한 메시지 추출  
        message = response_json['choices'][0]['message']  
        citaiton_list = message['context']['citations']

        # 역할(role)과 내용(content) 분리  
        role = message['role']  # 메시지의 역할 (예: assistant)  
        content = message['content']  # 메시지의 내용 (예: 남해 관광지 정보)   
        


        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content, citaiton_list
    else:
        return "", list()

In [33]:
import gradio as gr

with gr.Blocks() as demo:
    
    citation_state = gr.State([])

    def click_send(prompt, histories):
        print(prompt)
        content, citation_list = request_gpt(prompt=prompt)
        message_list = [{"role": "user", "content": prompt}, {"role": "assistant", "content": content}]
        histories.extend(message_list)
        
        print(histories)
        
        return histories, citation_list, ""
    
    gr.Markdown("# 🌟 성남시 동물 병원 찾기 🌟")
    
    gr.Markdown("### 💡 사용 방법")
    gr.Markdown("1. ✅ 프롬프트를 입력하고 '🚀 전송' 버튼을 클릭하세요.")
    gr.Markdown("2. ✅ GPT의 응답과 참조 목록이 아래에 표시됩니다.")
    gr.Markdown("3. ✅ **반드시 전화 후 방문하세요.**")
    
    gr.Markdown("### 🗺️ 성남시 지도")
    gr.HTML("""
        <iframe 
            src="https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d101420.5294234774!2d127.02937710083248!3d37.40421927664905!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!3m3!1m2!1s0x357ca7f1701220bf%3A0x50315666bce397d4!2z6rK96riw64-EIOyEseuCqOyLnA!5e0!3m2!1sko!2skr!4v1741929864508!5m2!1sko!2skr" 
            width="600" 
            height="450" 
            style="border:0;" 
            allowfullscreen="" 
            loading="lazy" 
            referrerpolicy="no-referrer-when-downgrade">
        </iframe>
    """)

    with gr.Row():
        chatbot = gr.Chatbot(label="🤖 나의 GPT", type="messages", scale=3)
    
        with gr.Column(scale=1):  
            @gr.render(inputs=[citation_state])
            def render_citations(citations):
                print(citations)
                for index in range(len(citations)):
                    content = citations[index]['content']
                    with gr.Tab("📚 참조 {}".format(index + 1)):
                        gr.Textbox(label="", value=content, lines=10, max_lines=10)
        
    with gr.Row():
        prompt_textbox = gr.Textbox(label="📝 프롬프트", placeholder="프롬프트를 입력하세요.", scale=6)
        send_button = gr.Button("🚀 전송", scale=1)
        
    send_button.click(click_send, inputs=[prompt_textbox, chatbot], outputs=[chatbot, citation_state, prompt_textbox])
    prompt_textbox.submit(click_send, inputs=[prompt_textbox, chatbot], outputs=[chatbot, citation_state, prompt_textbox])
    

    # 눈에 편안한 테마를 위한 CSS 스타일
    demo.css = '''
        body {
            background-color: #f4f4f4; /* 부드러운 회색 배경 */
            color: #333; /* 어두운 텍스트 색상 */
        }
        .gradio-container {
            background-color: #333333; /* 흰색 배경 */
            border-radius: 10px; /* 둥근 모서리 */
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1); /* 부드러운 그림자 */
        }
        .gradio-textbox {
            background-color: #e9ecef; /* 연한 회색 배경 */
            color: #333; /* 어두운 텍스트 색상 */
            border: 1px solid #ccc; /* 연한 회색 테두리 */
            border-radius: 5px; /* 둥근 모서리 */
            padding: 10px; /* 패딩 추가 */
        }
        .gradio-textbox:focus {
            border-color: #4CAF50; /* 포커스 시 테두리 색상 변경 */
            box-shadow: 0 0 5px rgba(76, 175, 80, 0.5); /* 포커스 시 그림자 추가 */
        }
        .gradio-button {
            background-color: #4CAF50; /* 버튼 배경색 */
            color: white; /* 버튼 텍스트 색상 */
            border: none; /* 테두리 제거 */
            border-radius: 5px; /* 둥근 모서리 */
            padding: 10px; /* 패딩 추가 */
        }
        .gradio-button:hover {
            background-color: #45a049; /* 버튼 호버 시 색상 변경 */
        }
    '''

demo.launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


[]
구미동 동물 병원 3개 추천해줘
<Response [200]>
[{'role': 'user', 'content': '구미동 동물 병원 3개 추천해줘'}, {'role': 'assistant', 'content': '구미동에 위치한 동물 병원 3개를 추천해 드립니다.\n\n1. **나라 동물병원**\n   - 주소: 경기도 성남시 분당구 구미동 205-1번지 오성프라자 113호\n   - 개업일: 2011-08-16\n   - 전화번호: 031-712-0707\n\n2. **현대 동물병원**\n   - 주소: 경기도 성남시 분당구 구미동 185-5번지 동아그린프라자 111호\n   - 개업일: 2012-03-23\n   - 전화번호: 031-718-7282\n\n3. **365 동물병원**\n   - 주소: 경기도 성남시 분당구 구미동 30번지 125호\n   - 개업일: 2017-04-04\n   - 전화번호: 031-718-0365\n\n이 정보는 각 병원의 개업일과 전화번호를 포함하고 있습니다. 필요하신 경우 추가 정보를 요청해 주세요!'}]
[{'content': '경기도 성남시 분당구 구미동 205-1번지 오성프라자 113호\n경기도 성남시 분당구 미금로 48, 오성프라자 113호 (구미동)\n나라 동물병원\n분당구\n2011-08-16\n구미동\n031-712-0707', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}, {'content': '경기도 성남시 분당구 구미동 185-5번지 동아그린프라자 111호\n경기도 성남시 분당구 성남대로 30, 111호 (구미동, 동아그린프라자)\n현대 동물병원\n분당구\n2012-03-23\n구미동\n031-718-7282', 'title': None, 'url': None, 'filepath': None, 'chunk_id': '0'}, {'content': '경기도 성남시 분당구 구미동 30번지 125호\n경기도 성남시 분당구 미금일로90번